<a href="https://colab.research.google.com/github/satyamnewale/Axios-Project/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from google.colab import files
uploaded = files.upload()


Saving messy_sales_data.csv to messy_sales_data.csv


In [ ]:
!pip install word2number
!pip install python-dateutil

In [71]:
import pandas as pd
from word2number import w2n
df = pd.read_csv('messy_sales_data.csv')
print(df.shape)
#cpnvert data into strings and remove extra spaces
str_cols = ["CustomerName","Gender","Category","City","Status"]
for c in str_cols:
  df[c]= df[c].astype(str).str.strip()
df["Category"] = df['Category'].str.lower()
df["City"]= df['City'].str.title().str.strip()
df["Status"]= df['Status'].str.lower()

gender_map = {"m":"M","male":"M","female":"F","f":"F"}
df['Gender'] = df['Gender'].str.lower().replace(gender_map)

#fix Quantity
def words_to_number(x):
    try:
        return w2n.word_to_num(x)
    except:
        return pd.NA
df["Quantity"] = df["Quantity"].astype(str).str.strip().str.lower().apply(words_to_number)
df["Quantity"] =pd.to_numeric(df["Quantity"],errors="coerce").astype("Int64")
df.loc[df["Quantity"] <= 0, "Quantity"] = pd.NA

#Age
def word_to_nos(x):
  try:
    return w2n.word_to_num(x)
  except:
    return pd.NA
df["Age"] = df["Age"].astype(str).str.strip().str.lower().apply(word_to_nos)

# age and CustmerID to int
df[['Age',"OrderID"]]=df[['Age',"OrderID"]].apply(
    lambda x: pd.to_numeric(x,errors='coerce').astype("Int64")
    )

#date
from dateutil import parser
def safe_parse(x):
  if pd.isna(x):
    return pd.NaT
  try:
    return parser.parse(str(x), dayfirst=True, fuzzy= True)
  except:
    return pd.NaT

df["Date"] = df["Date"].astype(str).apply(safe_parse)

# show rows where parse failed
bad_dates = df[df["Date"].isna()]
print("Unparsed dates:", bad_dates.shape[0])

#typos in status Category
df['Status'] = df['Status'].replace({"cancel":"cancelled","delvrd":"delivered"})
df['Category'] = df['Category'].replace({"furnitre":"furniture","clothng":"clothing","electrnics":"electronics"}).str.title()
# print(df[['Category','Status']].apply(lambda x: x.value_counts(dropna=True)).astype("Int64"))

# order id which are missing
max_id=df['OrderID'].max(skipna=True)
n_missing = df['OrderID'].isna().sum()
df.loc[df['OrderID'].isna(),'OrderID'] = range(max_id+1,max_id+1+n_missing)
df["OrderID"] = df["OrderID"].astype(int)

#amount simplify
# remove currency symbols and thousand separators; keep digits and dots and minus
df["Amount_clean"] = df["Amount"].astype(str).str.replace(r"[^0-9\.\-]", "", regex=True)
df["Amount_clean"] = pd.to_numeric(df["Amount_clean"], errors="coerce")
df["Amount"] = df["Amount_clean"]
df = df.drop(columns=["Amount_clean"])

#duplcates
print("Exact duplicates:", df.duplicated(subset=["OrderID","Date","Amount","CustomerName"]).sum())
df = df.drop_duplicates(subset=["OrderID","Date","Amount","CustomerName"],keep="first")
print("After dropping duplicates:", df.shape[0])

#missing dates and amount removed
df = df.dropna(subset=["Amount","Date"])

# missing quantity
df["Quantity"] = df["Quantity"].fillna(df["Quantity"].median())

#gender and age missing
df["Gender"] = df["Gender"].fillna("U")
median_age = df["Age"].median(skipna=True)
df["Age"] = df["Age"].fillna(median_age)

print(df.to_string())
print(df.isna().sum())

(1000, 10)
Unparsed dates: 180
Exact duplicates: 19
After dropping duplicates: 981
     OrderID  CustomerName Gender  Age       Date   Amount  Quantity     Category         City     Status
0       1102  David Miller      M   22 2024-05-12   900.00         2  Electronics        Delhi  delivered
1       1179      John Doe      F   22 2024-12-05  1500.75         3     Clothing       London  delivered
2       1092      John Doe      M   30 2024-12-05  1500.75         2    Furniture  Los Angeles    pending
3       1014  David Miller      F   25 2024-05-12  1200.00         1    Furniture  Los Angeles    pending
4       1106   Rahul Mehta      F   30 2024-05-13   700.00         2    Furniture       London    pending
5       1071    jane SMITH      M   30 2024-05-12   900.00         2     Clothing  Los Angeles  delivered
7       1020    jane SMITH      M   30 2024-12-05     1.00         2  Electronics       London  delivered
8       1102  David Miller      M   40 2024-05-12   800.50         3 